In [41]:
from io import StringIO

import pandas as pd
import math
import numpy as np

# import our own modules
import sys
sys.path.append('../src')

from pareto import pareto_parents
from parsing import Dataset
from time_function import generate_weight_profit_velocity

In [42]:
with open("a280-n279.txt", "r") as file:
    file_content = file.read()
    
dataset = Dataset.new(file_content)

In [43]:
print("Problem Name:", dataset.name)
print("Knapsack Type:", dataset.knapsack_type)
print("Nodes:", dataset.nodes)
print("Items:", dataset.items)

Problem Name: a280-TTP
Knapsack Type: ['bounded strongly corr']
Nodes: [<index:1,  x:288, y:149> <index:2,  x:288, y:129>
 <index:3,  x:270, y:133> <index:4,  x:256, y:141>
 <index:5,  x:256, y:157> <index:6,  x:246, y:157>
 <index:7,  x:236, y:169> <index:8,  x:228, y:169>
 <index:9,  x:228, y:161> <index:10,  x:220, y:169>
 <index:11,  x:212, y:169> <index:12,  x:204, y:169>
 <index:13,  x:196, y:169> <index:14,  x:188, y:169>
 <index:15,  x:196, y:161> <index:16,  x:188, y:145>
 <index:17,  x:172, y:145> <index:18,  x:164, y:145>
 <index:19,  x:156, y:145> <index:20,  x:148, y:145>
 <index:21,  x:140, y:145> <index:22,  x:148, y:169>
 <index:23,  x:164, y:169> <index:24,  x:172, y:169>
 <index:25,  x:156, y:169> <index:26,  x:140, y:169>
 <index:27,  x:132, y:169> <index:28,  x:124, y:169>
 <index:29,  x:116, y:161> <index:30,  x:104, y:153>
 <index:31,  x:104, y:161> <index:32,  x:104, y:169>
 <index:33,  x:90, y:165> <index:34,  x:80, y:157>
 <index:35,  x:64, y:157> <index:36,  x

In [44]:
def euclidean_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)


f = open("a280-n279.txt", "r")

#print(f.read())
number_of_cities=4
city_cordinates=np.zeros((number_of_cities, 2), dtype=np.float64)
dist_matrix=np.zeros((number_of_cities, number_of_cities), dtype=np.float64)
#storing cordinates of city nodes into a number of cities * 2 array
line=f.readline()
while(line.strip() != ''):
    
    if "NODE_COORD_SECTION" in line:
        for i in range(number_of_cities):
            line=f.readline()
            a = line.split()
            city_cordinates[i][0] = float(a[1].strip())
            city_cordinates[i][1] = float(a[2].strip())
        break
    line=f.readline()
    
for i in range(number_of_cities):
    for j in range(number_of_cities):
        if(i!=j):
           # print(city_cordinates[i])
            #print(city_cordinates[j])
            dist_matrix[i][j] = euclidean_distance(city_cordinates[i], city_cordinates[j])

print(dist_matrix)

[[ 0.         20.         24.08318916 32.984845  ]
 [20.          0.         18.43908891 34.17601498]
 [24.08318916 18.43908891  0.         16.1245155 ]
 [32.984845   34.17601498 16.1245155   0.        ]]


In [48]:
def generate_cities_and_items(Q, file):
    """
  This function generates and returns a random order in which cities are to be traversed,
  along with a binary array that decides which items must be put in the knapsack

  Parameters
  ----------
  Q : int
    The total capacity of the knapsack
  file : str
    The path to the dataset file


  Returns
  -------
  city_travel : np.array[int]
    The order of cities to be visited
  items_selected : np.array[int]
    A binary array which decides which items are to be picked

  """

    # reading the file
    dataset_file = open(file, 'r').read()
    heading1 = 'NODE_COORD_SECTION	(INDEX, X, Y): '
    heading2 = 'ITEMS SECTION	(INDEX, PROFIT, WEIGHT, ASSIGNED NODE NUMBER): '
    pos1 = dataset_file.find(heading1) + len(heading1)
    pos2 = dataset_file.find(heading2) + len(heading2)

    # unpacking the data
    cities_dataset = pd.read_csv(StringIO(dataset_file[pos1:pos2 - len(heading2)]), sep='\t')
    cities_dataset.columns = ['INDEX', 'X', 'Y']
    items_dataset = pd.read_csv(StringIO(dataset_file[pos2:]), sep='\t')
    items_dataset.columns = ['INDEX', 'PROFIT', 'WEIGHT', 'ASSIGNED NODE NUMBER']

    city_travel = np.random.choice(cities_dataset['INDEX'], size=len(cities_dataset), replace=False)
    weight_array = np.array(items_dataset['WEIGHT'])
    prob = Q / sum(weight_array)

    ####
    not_a_good_list = True
    items_select = None # initiate a packing list
    while (not_a_good_list):
        w = 0
        items_select = np.random.choice([0, 1], p=[1 - prob, prob], size=len(items_dataset))
        w = sum(weight_array * items_select)
        if w <= Q: not_a_good_list = False
        # the while loop keeps on generating the array z until the knapsack condition is not violated
    return city_travel, items_select
# Example usage:
Q = 100  # set your desired knapsack capacity
cities, items = generate_cities_and_items(Q, "a280-n279.txt")
print("Cities to Visit:", cities)
print("Items Selected:", items)

Cities to Visit: [ 47  68 241  77 127 175  59 103 102 125 259 270  93  98 249  66  58  64
  44 161 278  57 113 187 105  38 279 111 217  89  32  24 280  28 276 153
 210 244  84 200 268 235 136 209  25 122 214 201 182 269  41  43 188 277
 205 271 124 186  36 255  50 140 157 229  54 227 262 169 170 208 220  14
 101  13 218  72  73  75 231  99  60 246 178 117  12 104 151 265 171 267
  27 112  46 234  52 275  88 272 199 165   2  33  87 135 146 120 149 198
 152  67 213 110 141  19 247  61  76 129 143 118  55 237 138  62 168 260
  90 196  78  92 174   4  56 238 156  74 158  18  86 191 243 273   6 233
  17 144   9 223 263 115 172 107 194  91  40  97 197  95  51 159 216 230
   7  16 176 202 193 228 250 239   5  53 190  69 160 150 185  71  63 226
  79  35   3   8 254 248 133  81 114 257  42 236  49  83 139  80  45 126
 204 184 137  34  26 130 225 128 106  10 121  39  15 207 240 261  96 245
 116  23 264 119 131 180 212 192 183 215 132 266 195 164 203 179 206 211
 167 173  29 163 142 252 219 162 2

In [61]:


def calculate_travel_time(cities, distance_matrix, vmax, vmin, weights, profits, Q):
    """
    Calculates the total travel time and profit for a given route through cities, taking into account 
    the variation in velocity due to changes in the weight of the knapsack.

    Parameters:
    cities (list of str): List of cities in the order they will be visited.
    distance_matrix (dict): 2D dictionary with distances between each pair of cities.
    vmax (float): Maximum velocity of the thief.
    vmin (float): Minimum velocity of the thief.
    weights (dict): Dictionary with weights of items in each city.
    profits (dict): Dictionary with profits of items in each city.
    Q (float): Maximum capacity of the knapsack.

    Returns:
    tuple: A tuple of total travel time and total profit for the route.
    """

    # Validate the input parameters
    if vmax <= 0 or vmin <= 0 or vmax < vmin:
        raise ValueError("Invalid velocities: vmax and vmin must be positive, and vmax must be greater than vmin.")
    if any(city is None for city in cities) or not isinstance(distance_matrix, np.ndarray):
        raise ValueError("Cities must not contain None values, and distance_matrix must be a numpy array.")

    # Create a city index mapping
    city_index = {city: i for i, city in enumerate(cities)}

    # Initialize the variables for total travel time, current knapsack weight, profit, and velocity
    total_time, curr_wt_of_kns, curr_pro_of_kns, velocity = 0, 0, 0, vmin

    # Iterate through each city in the route
    for i in range(len(cities) - 1):
        # Determine the weight, profit, and velocity for the current city
        result = generate_weight_profit_velocity(vmax, vmin, weights[cities[i]], profits[cities[i]], curr_wt_of_kns, curr_pro_of_kns, Q)
        # If a valid combination is found, update the knapsack and velocity
        if result:
            curr_wt_of_kns, curr_pro_of_kns, velocity = result
        else:
            # If no valid combination is found, exit the loop
            break

        # Get the indices for the current and next city
        current_city_index = city_index[cities[i]]
        next_city_index = city_index[cities[i + 1]]

        # Calculate the time to travel to the next city and update the total travel time
        time_to_next_city = distance_matrix[current_city_index][next_city_index] / velocity
        total_time += time_to_next_city

    # If the last city processed successfully, calculate the time to return to the starting city
    if result:
        start_city_index = city_index[cities[0]]
        time_to_start_city = distance_matrix[current_city_index][start_city_index] / velocity
        total_time += time_to_start_city

    # The final total profit is the current profit in the knapsack
    total_profit = curr_pro_of_kns

    return total_time, total_profit

vmax = 10.0
vmin = 5.0
weights = {"city1": 5, "city2": 10, "city3": 8}  # Replace with your actual weights
profits = {"city1": 20, "city2": 30, "city3": 25}  # Replace with your actual profits
cities = ["city1", "city2", "city3"]  # Replace with your actual list of cities
dist_matrix = np.array([[0.0, 20.0, 24.08318916, 32.984845],
                        [20.0, 0.0, 18.43908891, 34.17601498],
                        [24.08318916, 18.43908891, 0.0, 16.1245155],
                        [32.984845, 34.17601498, 16.1245155, 0.0]])
Q = 50.0  

total_time, total_profit = calculate_travel_time(cities, dist_matrix, vmax, vmin, weights, profits, Q)
print("Total Travel Time:", total_time)
print("Total Profit:", total_profit)

Total Travel Time: [5.84390889]
Total Profit: [0]
